In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)
indicators = Indicators(alpaca_api)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    print('Losers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')

def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Ticker'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        
        cluster = indicators.cluster_prep(ticker)
        
        print("asdf:", cluster.iloc[-1])
        
        
        
        
        
        # print out some indicators here # TODO
        # also throw out shitty stocks
        
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\benjamin.beale\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Ticker     Last        Change      Pct Gain
–––––––––––––––––––––––––––––––––––––––––––––
asdf: time            2019-11-13 00:00:00-05:00
open                                 6.05
high                                 7.46
low                                  4.83
close                                 5.5
volume                           12942310
up_move                            4.6684
down_move                           -2.08
DMp                                4.6684
DMm                                     0
ticker                               NSYS
bb_up                             4.17644
bb_mid                            2.97389
bb_low                            1.77134
macd                             0.157449
signal                         -0.0110606
vwmacd                          0.0110708
vwsignal                       -0.0330936
mfi                               99.6479
stoch                             59.2524
vzo              

KeyboardInterrupt: 

In [3]:
ticker = "NSYS"

In [4]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [5]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [6]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format('–' * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for NSYS
–––––––––––––––––––––––––––––––––––––––––––––
Stocktwits:	positive	 score: 0.7784
Twitter:	positive	 score:	0.9531
Average:	0.86575


In [7]:
# alpaca_api.polygon.historic_trades(ticker, '2019-10-10', limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [8]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [9]:
stocktwits

{'response': {'status': 200},
 'symbol': {'id': 2903,
  'symbol': 'NSYS',
  'title': 'Nortech Systems Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 1807},
 'cursor': {'more': True, 'since': 183541446, 'max': 183537286},
 'messages': [{'id': 183541446,
   'body': '$NSYS watch out for the next VWAP cross. It is going to be violent.',
   'created_at': '2019-11-13T15:54:23Z',
   'user': {'id': 2847456,
    'username': 'rmaeso',
    'name': 'Rafael Maeso',
    'avatar_url': 'http://avatars.stocktwits.com/images/default_avatar_thumb.jpg',
    'avatar_url_ssl': 'https://s3.amazonaws.com/st-avatars/images/default_avatar_thumb.jpg',
    'join_date': '2019-10-07',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 0,
    'following': 6,
    'ideas': 15,
    'watchlist_stocks_count': 20,
    'like_count': 0,
    'plus_tier': '',
    'premium_room': ''},
   'source': {'id': 2269,
    'title': 'StockTwits Web',
    'url': 'https://stock

In [10]:
tweets 

"Free Strategy Talk: Time Weighted Float Analysis. Lets take a look at $NSYS chart. The numbers represent the minute… https://t.co/svt1kgjaDw\n\nwait 1-2h more when fiction meets reality, stock is going under $4 by end of the day $NSYS\n\nwondering whats the reason to buy up here, company reported earnings, stock doubled its market up.. are those earni… https://t.co/lJfNvvOebT\n\n$NSYS is running for the top. Here's what's happening! https://t.co/N0SE8CoQ7x https://t.co/oIsGMNYAI9\n\n#stocks #stockstowatch #StockMarket $FSLY $SPB $TVTY $NSYS $ENR $FCEL Stocks Moving With Catalysts On November 13 2… https://t.co/67bXuJY2Wa\n\nTop 5 gainers:\n $NSYS +94.77% https://t.co/GKLkyBoTKb\n $FCEL +21.82% https://t.co/uqP6vlZu76\n $LK +12.87%… https://t.co/zbwjXF7xJw\n\n$NSYS The Level II told me the story and I listened. I went long when buyers were heavily influencing this stock to… https://t.co/jzlWoXeWVy\n\nFollow gainers winners with our gainers scanners from PM to AH  Get notifications ever

In [11]:
tenquant_data

{'assets': 66779000.0,
 'bookvalue': 19635000.0,
 'comprehensiveincome': -1558000.0,
 'comprehensiveincomeattributabletononcontrollinginterest': 0.0,
 'comprehensiveincomeattributabletoparent': -1558000.0,
 'costofrevenue': 76594000.0,
 'currencycode': 'USD',
 'currentassets': 48260000.0,
 'currentliabilities': 25616000.0,
 'date': '2019-09-30',
 'dividendpayments': 0.0,
 'dividendyield': 0.0,
 'documenttype': '10-Q',
 'duration': 3,
 'equity': 19635000.0,
 'exchangegainslosses': 0.0,
 'extraordaryitemsgainloss': 0.0,
 'grossprofit': 8921000.0,
 'incomebeforeequitymethodinvestments': -1502000.0,
 'incomefromcontinuingoperationsaftertax': -1502000.0,
 'incomefromcontinuingoperationsbeforetax': -1380000.0,
 'incomefromequitymethodinvestments': 0.0,
 'incometaxexpensebenefit': 122000.0,
 'interestanddebtexpense': 780000.0,
 'liabilities': 47144000.0,
 'liabilitiesandequity': 66779000.0,
 'marketcap': 7306750,
 'netcashflow': 1008000.0,
 'netcashflowsfinancing': 2063000.0,
 'netcashflowsin